# Tests

In [4]:
import os
import sys
import logging
import requests
import json
import re
import pandas as pd

sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from lib.nanuq import Nanuq
from lib.gapp import Phenotips
from lib.gapp import BSSH
from lib.samplesheet import SampleSheet

nq    = Nanuq()
pho   = Phenotips()
bssh  = BSSH()

class Args:
    # Simulate an argparse instance
    #
    def __init__(self, run='LH00336_0006', level='info'):
        self.run   = run
        self.level = level

args = Args(level='debug')
vars(args)

{'run': 'LH00336_0006', 'level': 'debug'}

In [5]:
biosample = "25335"
biosampleid = bssh.get_biosample_id(biosample)
datasets    = bssh.get_dataset_id(biosampleid)
datasets

['ds.add451fb133443ca96c46b956102c1e7']

In [18]:
endpoint = '/v2/datasets/'
url      = bssh.server + endpoint
payload  = {'inputbiosamples': {biosampleid}} # , 'datasettypes': ''}
response = requests.get(url, headers=bssh.headers, params=payload)
response.raise_for_status

item0 = response.json().get('Items')[0]
print(f"{item0['Project']['Id']} {item0['Project']['Name']}")
print(json.dumps(item0["Project"], indent=2))

SyntaxError: f-string: unmatched '[' (3831853880.py, line 8)

In [ ]:
content = nq.get_samplesheet('LH00336_0009').text.splitlines()
print(content[1:10])

In [ ]:
# Get authorization token from Emedgene
# Please note - the Authorization header is only valid for a limited time, and
# expires after 8H. In that case, any request made with an expired token will 
# return a 403 code. To resolve, re-do the Login procedure to get a new token.
#
url      = "https://chusaintejustine.emedgene.com/api/auth/api_login/"
payload  = '{"username": "cqgc.bioinfo.hsj@ssss.gouv.qc.ca", "password": "3175CoteSainte-Catherine"}'
headers  = {'Content-Type': 'application/json'}
response = requests.request("POST", url, headers=headers, data=payload)
auth_header = response.json()["Authorization"]
auth_header


In [ ]:
# Get case for EMG634551172 (GM240123)
#
case_id = "EMG634551172"
get_test_response = requests.get(f'https://chusaintejustine.emedgene.com/api/test/{case_id}/', json={}, headers={'Authorization': auth_header})
get_test_response.json().keys()


In [ ]:
# Transfer files to aws
# --profile "emedgene" or "emedgene-eval"
aws s3 ls --profile emedgene-eval s3://cac1-prodca-emg-downloads/Ste_Justine_eval/upload